# Few-Shot Prompting

We will be using this technic by being explicit in the questions we wanted to ask,
for that we will verify that the api works first.

In [60]:
import openai
from openai import OpenAI
import json
import re
import helper.creden as cre
client = OpenAI(api_key=cre.OPENAI_API_KEY)
from helper.helper import parse_conversation, select_random_test, get_completion_from_messages

# widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

client = OpenAI(api_key=cre.OPENAI_API_KEY)
model_selection="gpt-4o-mini"

In [61]:
messages_verification=[{"role": "user", "content": "Are you working?"}]
result = get_completion_from_messages(messages = messages_verification, model = model_selection, temperature=0.8, max_tokens_values=4)
result

"Yes, I'm here"

Now that we have confirmed the API is working, we will conduct an experiment.

In [62]:
tests_interviews = {
                "topics": [
                    {
                    "name": "Predictive Modeling",
                    "example": "Predicting customer churn for a subscription service.",
                    "problem": "Identifying which customers are likely to stop using the service and why."
                    },
                    {
                    "name": "Recommendation Systems",
                    "example": "Suggesting products on an e-commerce platform.",
                    "problem": "Providing personalized recommendations to improve user engagement and sales."
                    },
                    {
                    "name": "Natural Language Processing (NLP)",
                    "example": "Sentiment analysis on customer reviews.",
                    "problem": "Understanding customer sentiment to improve product offerings or service quality."
                    },
                    {
                    "name": "Time Series Analysis",
                    "example": "Forecasting monthly sales for inventory planning.",
                    "problem": "Predicting demand accurately to avoid stockouts or overstocking."
                    },
                    {
                    "name": "Anomaly Detection",
                    "example": "Detecting fraudulent transactions in banking systems.",
                    "problem": "Identifying unusual patterns or behaviors that indicate fraud."
                    },
                    {
                    "name": "Image and Video Analytics",
                    "example": "Detecting defects in manufacturing products using image classification.",
                    "problem": "Automating quality control to reduce manual inspection errors."
                    },
                    {
                    "name": "Customer Segmentation",
                    "example": "Grouping customers by purchasing behavior.",
                    "problem": "Creating targeted marketing campaigns to boost conversion rates."
                    },
                    {
                    "name": "Optimization Models",
                    "example": "Optimizing delivery routes for a logistics company.",
                    "problem": "Minimizing fuel costs and delivery time."
                    },
                    {
                    "name": "A/B Testing",
                    "example": "Testing two website layouts to improve user conversion.",
                    "problem": "Determining which design leads to better engagement or sales."
                    },
                    {
                    "name": "Big Data Processing",
                    "example": "Analyzing clickstream data from millions of users on a streaming platform.",
                    "problem": "Deriving insights and recommendations from massive, unstructured datasets."
                    }
                ]
                }
# select a random test
random_topic = select_random_test(tests_interviews)
random_topic

{'name': 'Anomaly Detection',
 'example': 'Detecting fraudulent transactions in banking systems.',
 'problem': 'Identifying unusual patterns or behaviors that indicate fraud.'}

# Prompt generation

In [63]:
prompt_example = f"""
You are not allowed to talk about anything else apart from this topic.
We are making a datascience interview to a person, we need to just talk about that topic,
Try to continue to the next question once the answer its done. 

Here are the steps you need to follow:
Intro: Introduce yourself
Question_1: How can you resolve this projects {random_topic["example"]} this its kind of the problem {random_topic["problem"]} 
Question_2: How the dataset look like and how the we collect that data?   
Question_3: How do we implemente this projects, in real life, example, python,pytorch, sklearn, deployment, etc.  
Other_topics: Did you had any question for me!
Result: Generate a JSON with the results of each question (There are three questions) after it is answered. Assign a score from 0 to 2, where 2 indicates a correct answer, 1 for a partially correct response, and 0 for an incorrect one. Evaluate each question accordingly.

After that ask if something else its need it to help, if not just say good bye.
"""

context = [{'role':'system', 'content':prompt_example}]
context

[{'role': 'system',
  'content': '\nYou are not allowed to talk about anything else apart from this topic.\nWe are making a datascience interview to a person, we need to just talk about that topic,\nTry to continue to the next question once the answer its done. \n\nHere are the steps you need to follow:\nIntro: Introduce yourself\nQuestion_1: How can you resolve this projects Detecting fraudulent transactions in banking systems. this its kind of the problem Identifying unusual patterns or behaviors that indicate fraud. \nQuestion_2: How the dataset look like and how the we collect that data?   \nQuestion_3: How do we implemente this projects, in real life, example, python,pytorch, sklearn, deployment, etc.  \nOther_topics: Did you had any question for me!\nResult: Generate a JSON with the results of each question (There are three questions) after it is answered. Assign a score from 0 to 2, where 2 indicates a correct answer, 1 for a partially correct response, and 0 for an incorrect on

In [64]:
# Create text box for user input
user_input = widgets.Textarea(value='',
                              placeholder='Enter text here...',
                              description='User:',
                              rows=2,
                              layout=widgets.Layout(width='auto'))

# Create text area for conversation display
conversation_history = widgets.Textarea(value='',
                                        description='Chat:',
                                        rows=15,
                                        layout=widgets.Layout(width='auto'))

# Create button for user to click to send their message
send_button = widgets.Button(description='Send')

# Initial Question (without response)
initial_question = "We will start with the interview. Can you give me a brief introduction about yourself?"
conversation_history.value += 'assistant: ' + initial_question + '\n'
context.append({'role': 'assistant', 'content': initial_question})

# -------------------------------
def send_button_clicked(b):
    """Send content to the chat"""
    global context
    
    conversation_history.value += 'user: ' + user_input.value + '\n'
    context.append({'role': 'user', 'content': user_input.value})
    response = get_completion_from_messages(messages = context, model = model_selection, temperature=0.1, max_tokens_values=600)
    conversation_history.value += 'assistant: ' + response + '\n'
    context.append({'role': 'assistant', 'content': response})
    user_input.value = ''
    
# -------------------------------
send_button.on_click(send_button_clicked)

# -------------------------------
# Add keyboard shortcut for sending (Shift+Enter)
def handle_keypress(widget):
    if widget.value.endswith('\n'):
        send_button_clicked(None)
        
user_input.observe(handle_keypress, names='value')
# -------------------------------
# Display everything
display(conversation_history, user_input, send_button)

Textarea(value='assistant: We will start with the interview. Can you give me a brief introduction about yourse…

Textarea(value='', description='User:', layout=Layout(width='auto'), placeholder='Enter text here...', rows=2)

Button(description='Send', style=ButtonStyle())

# Review of the interaction

In [68]:
json_result = parse_conversation(conversation_history.value)
print(json.dumps(json_result, indent=2))

{
  "conversation": [
    {
      "role": "assistant",
      "content": "We will start with the interview. Can you give me a brief introduction about yourself?"
    },
    {
      "role": "user",
      "content": "My name its Karla"
    },
    {
      "role": "assistant",
      "content": "Nice to meet you, Karla! Let's dive into the first question.\n**Question 1:** How can you resolve the project of detecting fraudulent transactions in banking systems? This involves identifying unusual patterns or behaviors that indicate fraud."
    },
    {
      "role": "user",
      "content": "by cheking for anomalys using algos like tsne or knn to detects those outliers"
    },
    {
      "role": "assistant",
      "content": "Thank you for your response!\n**Score for Question 1:** 1 (partially correct response)\nNow, let's move on to the next question.\n**Question 2:** How does the dataset look like and how do we collect that data?"
    },
    {
      "role": "user",
      "content": "will had 

In [70]:
pattern = r'results in JSON format:\s*```json\s*({.*?})\s*```'

match = re.search(pattern, conversation_history.value, re.DOTALL)

if match:
    extracted_data = match.group(1)
    print(f"Extracted Data:\n{extracted_data}")
else:
    print("No match found.")

Extracted Data:
{
  "results": {
    "Question_1": {
      "answer": "by checking for anomalies using algorithms like t-SNE or KNN to detect those outliers",
      "score": 1
    },
    "Question_2": {
      "answer": "will have a lot of numerical values",
      "score": 1
    },
    "Question_3": {
      "answer": "we deploy it in the cloud and monitored using Weights and Biases",
      "score": 1
    }
  }
}


After reviewing, some questions may not be entirely clear, and adding examples could improve clarity. Further refinement is still needed.